In [21]:
import pandas as pd
import numpy as np
import re

In [2]:
df1 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-01.txt.gz", error_bad_lines=False, sep="\t")
df2 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-02.txt.gz", error_bad_lines=False, sep="\t")
df3 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-03.txt.gz", error_bad_lines=False, sep="\t")
df4 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-04.txt.gz", error_bad_lines=False, sep="\t")
df5 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-05.txt.gz", error_bad_lines=False, sep="\t")
df6 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-06.txt.gz", error_bad_lines=False, sep="\t")
df7 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-07.txt.gz", error_bad_lines=False, sep="\t")
df8 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-08.txt.gz", error_bad_lines=False, sep="\t")
df9 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-09.txt.gz", error_bad_lines=False, sep="\t")
df10 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-10.txt.gz", error_bad_lines=False, sep="\t")
df = df1.append([df2,df3,df4,df5,df6,df7,df8,df9,df10], ignore_index=True)

/Users/jarana/anaconda3/envs/recsys2019/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
len(df)

36389567

In [8]:
df = df.drop_duplicates(['AnonID', 'Query'], keep='first')

In [ ]:
# TODO drop row with www. .com .uk .us .edu

In [9]:
len(df)

17151348

In [5]:
df.columns

Index(['AnonID', 'Query', 'QueryTime', 'ItemRank', 'ClickURL'], dtype='object')

In [ ]:
how to make tulle butterflies	

In [14]:
df[df['AnonID']== 65534]

,AnonID,Query,QueryTime,ItemRank,ClickURL
25668,65534,sharon'spearls.com,2006-03-01 05:41:21,NaN,NaN
25669,65534,sharonspearl.com,2006-03-01 05:41:44,NaN,NaN
25671,65534,jennifer boyd columbia university,2006-03-03 06:35:10,NaN,NaN
25672,65534,pearlery,2006-03-03 06:38:03,NaN,NaN
25673,65534,pearllery,2006-03-03 06:38:12,NaN,NaN
25674,65534,special education assessor positions,2006-03-03 06:42:02,NaN,NaN
25675,65534,bulging disc,2006-03-03 23:29:33,2,http://www.spineuniverse.com
25676,65534,pdi.edu,2006-03-04 05:52:23,NaN,NaN
25677,65534,professional development institute,2006-03-04 05:52:45,2,http://www.webteaching.com
25678,65534,mortgage calculator,2006-03-04 06:19:32,2,http://www.bankrate.com


In [54]:
# '|'.join(["send", "group", "email"]
df[df['Query'].str.lower().str.contains("how to", na=False)]

,AnonID,Query,QueryTime,ItemRank,ClickURL
754,2722,how to make tulle butterflies,2006-04-12 15:05:40,3,http://www.butterflybreeders.com
990,2722,how to remove adware from computer,2006-05-21 15:23:09,1,http://www.microsoft.com
6619,8873,how to know when menopause is over,2006-03-27 15:13:15,1,http://www.ehow.com
7904,13373,how to paint concrete floor,2006-04-21 10:11:57,1,http://interiordec.about.com
9253,15565,pageant hair extentions how tos,2006-04-10 02:28:50,NaN,NaN
11969,26111,how to uninstall a kitchen facuit,2006-05-18 19:15:38,NaN,NaN
11970,26111,how to uninstall a kitchen faucet,2006-05-18 19:15:48,NaN,NaN
12508,29859,how to change timing belts,2006-03-18 09:28:38,1,http://www.miata.net
12509,29859,how to change hyndia timing belts,2006-03-18 09:30:27,NaN,NaN
12832,29894,how to get free web on nextel phones,2006-05-24 19:41:01,NaN,NaN


In [51]:
df[df.AnonID == 5581588]

,AnonID,Query,QueryTime,ItemRank,ClickURL
23852185,5581588,fcmcclerk,2006-03-02 14:01:49,1,http://www.fcmcclerk.com
23852187,5581588,palace theater columbus ohio,2006-03-23 10:03:10,1,http://www.tickco.com
23852189,5581588,ohio theater,2006-03-23 10:10:06,2,http://www.capa.com
23852190,5581588,cpc.logistics.com,2006-03-28 16:55:22,4,http://www.colder.com
23852191,5581588,merci.com,2006-03-28 17:00:09,1,http://www.imdb.com
23852192,5581588,top warehouses in columbus ohio,2006-03-29 10:39:45,NaN,NaN
23852193,5581588,warehousing columbus ohio,2006-03-29 10:40:26,6,http://www.wbns10tv.com
23852195,5581588,www.osumedcenter.edu,2006-03-29 10:49:47,1,http://jobs.osu.edu
23852196,5581588,build a bear workshop inc.,2006-03-29 17:01:05,7,http://www.forbes.com
23852197,5581588,federal form eeo-1,2006-03-30 17:01:15,2,http://www.eeoc.gov


In [15]:
wiki = pd.read_csv("data/wikihowSep.csv", sep=",", error_bad_lines=False)

In [40]:
how = set([])
for i in wiki.title.str.lower().unique().tolist():
    how.add(regex.sub(' ', str(i)).replace("how to","")[1:])

In [42]:
how

{'get strong legs like a horse',
 'improve your technical writing skills',
 'deal with sensory processing disorder as a teen',
 'select a hand tied bouquet for your wedding flowers',
 'advertise wikihow as an editor',
 'find a easy and free typing program',
 'conduct the burning money experiment',
 'make code names',
 'send group email ',
 'dress when you are fat',
 'buy stocks',
 'archive email ',
 'burn itunes music to mp  cd for stereos and cd players',
 'copy cd to dvd',
 'download an xbox     game ',
 'get a treecko in pok mon emerald',
 'be a popular girl on moviestarplanet',
 'create dark art using photoshop',
 'teach a stubborn dog to sit down ',
 'get full dragon on runescape',
 'make rum punch ',
 'tell if someone added you on snapchat',
 'make pav bhaji  mumbai ',
 'view weekly fitbit stats ',
 'replace parts on the nikon d   ',
 'take telephone messages',
 'recover from back to work blues ',
 'clean your bedroom from top to bottom',
 'hunt turkeys ',
 'make crispy honey chi